In [1]:
from statsmodels.stats.power import TTestIndPower
import math
from scipy.stats import norm

# числовий

In [2]:
def ab_test_duration(sigma, delta, alpha=0.05, power=0.8, daily_users=50, alternative='two-sided'):
    """
    Розрахунок тривалості A/B тесту для метрики з нормальним розподілом.
    
    sigma: стандартне відхилення метрики
    delta: мінімальна різниця, яку хочемо виявити
    alpha: рівень значущості (тип I помилка)
    power: потужність тесту (1 - beta)
    daily_users: середня кількість користувачів на день
    alternative: тип альтернативної гіпотези ('two-sided', 'larger', 'smaller')
        'two-sided' — двосторонній тест. Перевіряє, чи є будь-яка різниця між групами (вище чи нижче).
        'larger' — односторонній тест, перевіряє, чи нова група більше за контроль.
        'smaller' — односторонній тест, перевіряє, чи нова група менше за контроль.
    """
    analysis = TTestIndPower()
    
    # effect_size = різниця / стандартне відхилення
    effect_size = delta / sigma
    
    n_per_group = analysis.solve_power(effect_size=effect_size,
                                       alpha=alpha,
                                       power=power,
                                       alternative=alternative)
    
    n_per_group = math.ceil(n_per_group)
    days_needed = math.ceil(n_per_group / daily_users)
    
    return n_per_group, days_needed

In [3]:
sigma = 20          # стандартне відхилення депозиту
delta = 5           # мінімальна різниця, яку хочемо виявити
daily_users = 50    # користувачів на день

n, days = ab_test_duration(sigma, delta, daily_users=daily_users)
print(f"Користувачів на групу: {n}")
print(f"Тривалість тесту (днів): {days}")

Користувачів на групу: 253
Тривалість тесту (днів): 6


Потрібно 253 користувача на кожну групу аби за 6 днів побачити зміни в 5 доларів в середньому депозиті

# біноміальний

In [4]:
def ab_test_proportion(
    p_hat,
    delta,
    alpha=0.05,
    power=0.8,
    daily_users=50,
    alternative='two-sided'
    ):
    """
    Розрахунок кількості користувачів та тривалості A/B тесту для пропорцій.
    p_hat: базова конверсія (наприклад 0.1)
    delta: мінімальний абсолютний ефект (наприклад 0.02)
    alpha: рівень значущості (тип I помилка)
    power: потужність тесту (1 - beta)
    daily_users: середня кількість користувачів на день (на групу)
    alternative: тип гіпотези ('two-sided', 'larger', 'smaller')
    """

    # z_alpha залежить від типу тесту
    if alternative == 'two-sided':
        z_alpha = norm.ppf(1 - alpha / 2)
    else:
        z_alpha = norm.ppf(1 - alpha)

    z_beta = norm.ppf(power)

    # формула для пропорцій
    n_per_group = (2 * (z_alpha + z_beta) ** 2 * p_hat * (1 - p_hat)) / (delta ** 2)

    n_per_group = math.ceil(n_per_group)
    days_needed = math.ceil(n_per_group / daily_users)

    return n_per_group, days_needed

Опис

Ви є продакт-менеджером застосунку для стримінгу музики під назвою MelodyFlow. Застосунок дозволяє користувачам створювати плейлисти, відкривати нову музику та слухати пісні за їхнім бажанням.
У застосунку є два типи підписок: безкоштовна (з рекламою) та преміум (без реклами). Команда розробила нову функцію під назвою «Розумні рекомендації плейлистів», яка використовує штучний інтелект для автоматичного створення персоналізованих плейлистів на основі історії прослуховувань користувача.
Очікується, що ця функція покращить залученість і утримання користувачів, спрощуючи пошук музики, яка їм подобається.

Дизайн тесту:

- Контрольна група (A): Користувачі не бачитимуть нову функцію «Розумні рекомендації плейлистів».
- Тестова група (B): Користувачі матимуть доступ до нової функції «Розумні рекомендації плейлистів».
Поточні дані:
- Щоденна активність користувачів (DAU): 20% від загальної кількості користувачів заходять щодня.
- Загальна кількість користувачів: 1 000 000 користувачів (600 000 безкоштовних, 400 000 преміум).
- Середня тривалість сесії: 15 хвилин за сесію.
- Базовий рівень утримання (7-денний): 41%.
- ARPPU за останній місяць: 25$

Завдання

Розрахувати розмір вибірки для біноміальної метрики.

Гіпотеза: Запровадження функції «Розумні рекомендації плейлистів» збільшить показник утримання клієнтів на 7-й день.

In [9]:
p_hat = 0.41          
# Базова конверсія (7-денне утримання)
# Вихідні дані з опису: поточний рівень утримання 41% (0.41)

delta = 0.029         
# Мінімальний абсолютний ефект (MDE)
# Ми очікуємо приріст на ~7% від базового рівня:
# 0.41 * 0.07 ≈ 0.0287, округляємо до 0.029
# Це означає, що ми хочемо зафіксувати збільшення утримання до 43.9%

alpha = 0.05          
# Рівень значущості (тип I помилка)
# Класичне значення: ймовірність помилково відкинути нульову гіпотезу = 5%

power = 0.8           
# Потужність тесту (1 - beta)
# Ймовірність виявити ефект, якщо він є, = 80%

daily_users = 0.2 * 1000_000 / 2  
# Середня кількість користувачів на день на групу
# - DAU = 20% від загальної кількості користувачів (1_000_000 * 0.2 = 200 000)
# - Тест ділиться на 2 групи (контрольна та тестова), тому на групу: 200 000 / 2 = 100 000

In [10]:
n, days = ab_test_proportion(
    p_hat=p_hat,
    delta=delta,
    alpha=alpha,
    power=power,
    daily_users=daily_users,
    alternative='two-sided'
)

print(f"Потрібно користувачів на групу: {n}")
print(f"Тривалість тесту: {days} днів")

Потрібно користувачів на групу: 4516
Тривалість тесту: 1 днів
